In [1]:
data_var = '2023-12-22'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9513,2023-12-22,Brasil Nbb,19:00,Minas,Pato,1.09,6.68,158.5,1.88,1.78,-15.5,2.10,1.66,nLQRxYdk,0.917431,0.149701,0.531915,0.561798,0.067132,0.4,0.8,NaN,NaN,106.386,24.221842,0.227679,211.872,44.004143,0.207692,101.266,240.464,94.64,224.07,0.0,0.0,0.0,0.0,1.017433,0.038640,0.165493,-1.48,-0.296,-0.304054,0.0,0.0,0.0,-2.66,-0.532,-10.676692,0.0,0.0,0.0
9514,2023-12-22,Espanha Acb,15:00,Bilbao,Basquet Girona,1.40,2.98,163.5,1.83,1.93,-7.5,2.08,1.71,lGavPd3g,0.714286,0.335570,0.546448,0.518135,0.049856,0.0,0.0,NaN,NaN,233.486,144.813310,0.620223,266.230,223.210679,0.838413,0.000,0.000,93.16,252.30,0.0,0.0,0.0,0.0,0.510150,0.037612,0.138063,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
9515,2023-12-22,Espanha Acb,16:30,MoraBanc Andorra,Gran Canaria,2.40,1.58,165.5,1.86,1.92,2.5,1.99,1.78,2J3rOGIa,0.416667,0.632911,0.537634,0.520833,0.049578,0.0,0.0,NaN,NaN,348.332,205.450604,0.589813,176.380,100.071440,0.567363,0.000,0.000,249.48,344.35,0.0,0.0,0.0,0.0,0.291371,0.022448,0.078776,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
9516,2023-12-22,Espanha Acb,17:00,Tenerife,Zaragoza,1.22,4.39,160.5,1.80,1.99,-10.5,2.00,1.77,dt3nNzY5,0.819672,0.227790,0.555556,0.502513,0.047463,0.0,0.0,NaN,NaN,185.408,98.218623,0.529743,239.216,109.785423,0.458938,0.000,0.000,104.40,154.88,0.0,0.0,0.0,0.0,0.799119,0.070897,0.086278,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
9517,2023-12-22,Espanha Acb,17:30,Murcia,Unicaja,1.95,1.86,163.5,1.84,1.94,-1.5,2.08,1.71,zX1jMfmC,0.512821,0.537634,0.543478,0.515464,0.050455,0.4,0.6,NaN,NaN,143.460,46.539784,0.324409,174.318,155.187608,0.890256,147.702,203.608,119.35,120.06,0.0,0.0,0.0,0.0,0.033407,0.037413,0.138063,1.87,0.374,2.540107,0.0,0.0,0.0,6.31,1.262,0.681458,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9667,2023-12-22,Hungria Nacional,14:00,Zalakeramia,Budapesti Honved Se,1.30,3.31,163.5,1.86,1.80,-8.5,2.00,1.72,OfLjiWek,0.769231,0.302115,0.537634,0.555556,0.071346,0.0,0.0,NaN,NaN,293.508,260.815299,0.888614,178.006,29.086747,0.163403,0.000,0.000,105.12,196.31,0.0,0.0,0.0,0.0,0.616609,0.023184,0.106446,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
9668,2023-12-22,Japão Liga Feminina W,07:00,Haneda Vickies F,Denso Iris F,11.60,1.02,137.5,1.82,1.85,22.5,2.02,1.71,WlCIxT60,0.086207,0.980392,0.549451,0.540541,0.066599,0.0,0.0,NaN,NaN,354.256,271.460459,0.766283,NaN,NaN,NaN,0.000,0.000,174.22,66.56,0.0,0.0,0.0,1.0,1.185609,0.011560,0.117535,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
9669,2023-12-22,Rússia Premier League Feminina,11:00,Nadezhda F,MBA Moscow F,1.65,2.15,136.5,1.85,1.85,-4.5,2.10,1.62,bJA0Qo5C,0.606061,0.465116,0.540541,0.540541,0.071177,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,87.04,68.48,0.0,0.0,0.0,1.0,0.186081,0.000000,0.182479,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
9670,2023-12-22,Rússia Premier League Feminina,11:00,UMMC Ekaterinburg F,Novosibirsk F,1.01,28.00,141.5,1.85,1.85,-26.5,2.02,1.69,x4MiS7z0,0.990099,0.035714,0.540541,0.540541,0.025813,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,72.60,196.68,0.0,0.0,0.0,0.0,1.315740,0.000000,0.125793,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
6,21:00,Eua Nba,Philadelphia 76ers,Toronto Raptors,227.5,1.83,0.9999,Over
7,21:30,Eua Nba,Brooklyn Nets,Denver Nuggets,229.5,1.86,1.0000,Over
13,16:30,Europa Euroliga,Bayern,Monaco,158.5,1.91,0.8376,Over
14,16:45,Europa Euroliga,Milano,Panathinaikos,158.5,1.91,0.8376,Over
